In [3]:
import os
from os import path
import pandas as pd
import numpy as np
import scipy.stats

# Import other necessary libraries

In [ ]:
# Set path
path = "/n/hausmann_lab/lab/atlas/bustos_yildirim/atlas_stata_cleaning/src"
data_path = os.path.join(path, "data")
program_path = os.path.join(path, "clean", "comtrade_reads_zip.py")
rfile_path = os.path.join(raw_data_path, f"{classification}_{year}.zip")


# Define years
startyear = 2015
finalyear = 2015

In [ ]:
def load_data(classification, year):


In [ ]:
# Function to clean data
def cleandata(classification, year):
    print(f"   > {year} and classification = {classification}")

    # Change directory to specific classification raw data and initialize paths
    raw_data_path = os.path.join(path, f"data/raw/{classification}_raw")
    program_path = os.path.join(path, "clean", "comtrade_reads_zip.py")
    rfile_path = os.path.join(raw_data_path, f"{classification}_{year}.zip")

    # Load the temporary data
    df = pd.read_stata(
        os.path.join(path, "data/processed/prepped_{classification}_{year}.dta")
    )

    # Data manipulation based on 'classification'
    # This is a simplification, adapt based on actual logic and conditions in Stata script
    if classification in ["H0", "HS"]:
        # Add "00" prefix to 'commoditycode' where the length of 'commoditycode' is 4 and 'agglevel' is 6
        df["commodity_code"] = np.where(
            (df["commodity_code"].str.len() == 4) & (df["agglevel"] == 6),
            "00" + df["commodity_code"],
            df["commodity_code"],
        )
        # Add "0" prefix to 'commoditycode' where the length of 'commoditycode' is 5 and 'agglevel' is 6
        df["commodity_code"] = np.where(
            (df["commodity_code"].str.len() == 5) & (df["agglevel"] == 6),
            "0" + df["commodity_code"],
            df["commodity_code"],
        )
        df["reporter_ansnoclas"] = df.trade_value.where(
            (df.partner_iso == "ANS")
            & (df.agglevel == 4)
            & (df.commodity_code.str.slice(0, 4) == "9999")
        )

    elif classification in ["S1", "S1", "ST"]:
        # Add "00" prefix to 'commoditycode' where the length of 'commoditycode' is 2 and 'agglevel' is 4
        df["commodity_code"] = np.where(
            (df["commodity_code"].str.len() == 2) & (df["agglevel"] == 4),
            "00" + df["commodity_code"],
            df["commodity_code"],
        )
        # Add "0" prefix to 'commoditycode' where the length of 'commoditycode' is 3 and 'agglevel' is 4
        df["commodity_code"] = np.where(
            (df["commodity_code"].str.len() == 3) & (df["agglevel"] == 4),
            "0" + df["commodity_code"],
            df["commodity_code"],
        )
        df["reporter_ansnoclas"] = df.trade_value.where(
            (df.partner_iso == "ANS")
            & (df.agglevel == 4)
            & (df.commodity_code == "9310")
        )

    # handles Germany (reunification) and Russia
    # drop if reporter and partner are DEU and DDR, trading with itself
    df = df[~((df["reporter_iso"] == "DEU") & (df["partner_iso"] == "DDR"))]
    df = df[~((df["reporter_iso"] == "DDR") & (df["partner_iso"] == "DEU"))]
    df.loc[df["partner_iso"].isin(["DEU", "DDR"]), "partner_iso"] = "DEU"
    df.loc[df["reporter_iso"].isin(["DEU", "DDR"]), "reporter_iso"] = "DEU"
    df.loc[df["partner_iso"].isin(["RUS", "SUN"]), "partner_iso"] = "RUS"
    df.loc[df["reporter_iso"].isin(["RUS", "SUN"]), "reporter_iso"] = "RUS"

    # compress
    # collapse (sum) tradevalue reporter_ansnoclas , by( year tradeflow agglevel reporter_iso partner_iso )
    df_collapsed = (
        df.groupby(["year", "tradeflow", "agglevel", "reporter_iso", "partner_iso"])
        .agg({"tradevalue": "sum", "reporter_ansnoclas": "sum"})
        .reset_index()
    )
    # recast float tradevalue reporter_ansnoclas, force
    df_collapsed["tradevalue"] = df_collapsed["tradevalue"].astype("float")
    df_collapsed["reporter_ansnoclas"] = df_collapsed["reporter_ansnoclas"].astype(
        "float"
    )

    # Return the cleaned DataFrame
    return df_collapsed


# Loop over years and classifications, calling the cleandata function
for year in range(startyear, finalyear + 1):
    print(f"> Doing year = {year}")

    # Example classifications processing, add as necessary
    if 1976 <= year < 1995:
        classification = "S2"
        cleaned_data = cleandata(classification, year)
        # Save cleaned data to file, replace `trade_S2` with actual path or filename
        cleaned_data.to_csv(os.path.join(path, f"Totals_raw_{year}.csv"), index=False)

    # Continue with other classification and year conditions

# After processing all years and classifications, you might want to merge, manipulate,
# and analyze the cleaned datasets similarly to how it's done in the Stata script.